In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import matplotlib.patches as patches
from descartes.patch import PolygonPatch
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

import networkx as nx
import pygraphviz
import pydot
from networkx.drawing.nx_agraph import graphviz_layout

import random
import pandas as pd
import napari

import random
from shapely.geometry import Point,MultiPoint,MultiPolygon,Polygon,box,LineString
from shapely import affinity
import re

from shapely import geometry
import pickle
import time

import ipywidgets as widgets
import ipywidgets.widgets.interaction
from ipywidgets import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)

from copy import deepcopy
import magicgui as mg
from magicgui import magicgui


import skimage.measure as measure
from dask_image.imread import imread
import src

%load_ext autoreload

In [2]:
path_to_uj_strix="/home/esb/Projects/uJ/local/current/uJ_src_SingleChan/python/"
path_to_uj_trillion='/home/charly/Lab/Projects/uJ/local/current/uJ_src_SingleChan/python/'
path_to_uj_trillion_online='/home/charly/Lab/Projects/uJ/uJ_src_online/python/'
path_to_uj=path_to_uj_trillion_online
#path_to_uj=path_to_uj_trillion

sys.path.insert(0, path_to_uj)


In [3]:
%autoreload 2
from readroi import *
from DataManagers import *
from TrackingFunctions import *
from PlottingFunctions import *
from AnalysisFunctions import *
from CorrectorFunctions import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!
TrackingFunctions... 1loaded!
PlottingFunctions... loaded!
Analysis Functions loaded
CorrectorFunctions... loaded!


In [4]:

uj_data_path_trillion="/home/charly/Lab/Projects/uJ/uJ_data/"
uj_data_path_trillion_online="/home/charly/Lab/Projects/uJ/uJ_data_online/"
uj_data_path=uj_data_path_trillion
#uj_data_path=uj_data_path_trillion_online

expeLabel="pBGT-AMP-Pulse"

rootDir=uj_data_path+"pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/"

full_trap_list="xy01,xy02,xy03,xy11,xy12,xy13,xy14,xy20,xy21,xy22,xy23,xy25,xy26,xy27,xy28,xy29,xy30,xy33,xy34,xy35,xy36,xy37,xy39,xy40,xy41,xy42,xy43,xy44,xy45,xy46,xy47".split(",")



print(rootDir)


channels=["DIC","GFP","DsRed"]
channels_color=["gray","green","red"]
vis=[False,True,True]
vis=[False,False,False]
seg_chan=channels[1]



frame2min=10

figxlim=640
figylim=512
flims=(figxlim,figylim)


frame_experiment_start=20;
frame_experiment_end=44
frame_signal_start=26
frame_signal_end=34
frame_norm_start=20
frame_norm_end=25
frame_analysis_end=44

n_frames=frame_experiment_end-frame_experiment_start+1

t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min

frames=range(int(frame_experiment_start),int(frame_experiment_end+1))
frames_list=[x for x in frames]
times=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]

data_type="normData"


dirNameDataLineages=rootDir+'data/lineages/'

dirNameDataNapariLabels=rootDir+'data_napari_labels/'
if not os.path.exists(dirNameDataNapariLabels):
    print("making dir napari labels...")
    os.makedirs(dirNameDataNapariLabels)

dirNameDataCellsTrackedCorrected=rootDir+'data_cells_tracked_corrected/'
if not os.path.exists(dirNameDataCellsTrackedCorrected):
    print("making dir Cells Tracked Corrected...")
    os.makedirs(dirNameDataCellsTrackedCorrected)

    
dirNameDataLineagesCorrected=rootDir+'data/lineages_corrected/'
if not os.path.exists(dirNameDataLineagesCorrected):
    print("making dir lineages corrected...")
    os.makedirs(dirNameDataLineagesCorrected)

#Prepare file system
dataPath="%sdata/GFP/"%rootDir


trap_masks_path=[]
trap_tif_path=[]
for pos in full_trap_list:
    tif_path=rootDir+"data_raw/"+pos+"/"
    trap_tif_path.append(tif_path)
    masks_path=rootDir+"data_masks/"+pos+"/"+seg_chan+"/"
    trap_masks_path.append(masks_path)
    
    
    




/home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/


## Chose correction trap

### Do not run this cell to perform calculations for all traps
### For trap correction, please choose one

In [5]:
current_trap="All"
current_trap="xy02"
trap_listw=["All"]+full_trap_list
trap_menu=widgets.Dropdown(options=trap_listw,value=current_trap,description='Trap:',)
trap_selection_widget=interactive(get_trap, sel=trap_menu,this_trap_list=fixed(full_trap_list))
output = trap_selection_widget.children[-1]
trap_selection_widget

interactive(children=(Dropdown(description='Trap:', index=2, options=('All', 'xy01', 'xy02', 'xy03', 'xy11', '…

## Prepare data

### Load tracked cells

In [6]:
max_frames=-1

trap_tracked_cells=[]
trap_tracked_cells_corrected=[]
trap_corrected_list=[]
for trap in trap_list:
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+"/"+data_type+"/"
    
    dirNameTRACKEDCELLS_Corrrected=dirNameDataCellsTrackedCorrected+trap+"/"+data_type+"/"
    iscorrected=os.path.exists(dirNameTRACKEDCELLS_Corrrected)
    trap_corrected_list.append(iscorrected)
    if(iscorrected):
        print("Loading corrected cells !")
        tracked_cells_corrected=load_cells(dirNameTRACKEDCELLS_Corrrected,max_frames)
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    else:
        print("Loading original cells !")
        tracked_cells=load_cells(dirNameTRACKEDCELLS,max_frames)
        trap_tracked_cells.append(tracked_cells)
    
        tracked_cells_corrected=tracked_cells.copy()
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    

Loading corrected cells !
0: Loading pBGT-AMP-Pulse_xy02_020.pkl
1: Loading pBGT-AMP-Pulse_xy02_021.pkl
2: Loading pBGT-AMP-Pulse_xy02_022.pkl
3: Loading pBGT-AMP-Pulse_xy02_023.pkl
4: Loading pBGT-AMP-Pulse_xy02_024.pkl
5: Loading pBGT-AMP-Pulse_xy02_025.pkl
6: Loading pBGT-AMP-Pulse_xy02_026.pkl
7: Loading pBGT-AMP-Pulse_xy02_027.pkl
8: Loading pBGT-AMP-Pulse_xy02_028.pkl
9: Loading pBGT-AMP-Pulse_xy02_029.pkl
10: Loading pBGT-AMP-Pulse_xy02_030.pkl
11: Loading pBGT-AMP-Pulse_xy02_031.pkl
12: Loading pBGT-AMP-Pulse_xy02_032.pkl
13: Loading pBGT-AMP-Pulse_xy02_033.pkl
14: Loading pBGT-AMP-Pulse_xy02_034.pkl
15: Loading pBGT-AMP-Pulse_xy02_035.pkl
16: Loading pBGT-AMP-Pulse_xy02_036.pkl
17: Loading pBGT-AMP-Pulse_xy02_037.pkl
18: Loading pBGT-AMP-Pulse_xy02_038.pkl
19: Loading pBGT-AMP-Pulse_xy02_039.pkl
20: Loading pBGT-AMP-Pulse_xy02_040.pkl
21: Loading pBGT-AMP-Pulse_xy02_041.pkl
22: Loading pBGT-AMP-Pulse_xy02_042.pkl
23: Loading pBGT-AMP-Pulse_xy02_043.pkl
24: Loading pBGT-AMP-Pul

### Load lineages files

In [7]:
trap_df_lineages_all=[]
trap_df_lineages_corrected_all=[]

for i,trap in enumerate(trap_list):
    
    if(trap_corrected_list[i]):
        print("Loading corrected lineages !")
        lineagesDataFile='%s%s_%s_lineages_all.csv'%(dirNameDataLineagesCorrected, expeLabel, trap)
        df_lineagesC_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
    else:
        lineagesDataFile='%s%s_%s_lineages_all.csv'%(dirNameDataLineages, expeLabel, trap)
        df_lineages_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_all.append(df_lineages_all)
        print("Loading original lineages !")
        df_lineagesC_all=df_lineages_all.copy()
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
#df_lineages=df_lineages_all

Loading corrected lineages !
Loading 540 lineages from /home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data/lineages_corrected/pBGT-AMP-Pulse_xy02_lineages_all.csv


### Add geometric info to dataframe

In [8]:
trap_df_lineages_geoms=[]

for i,trap in enumerate(trap_list):
    if(trap_corrected_list[i]):
        tracked_cells=trap_tracked_cells_corrected[i]
        df_lineages=trap_df_lineages_corrected_all[i]
    else:
        tracked_cells=trap_tracked_cells[i]
        df_lineages=trap_df_lineages_all[i]
    trap_newDF_list=[]
    for row in df_lineages.itertuples(index=False):
        this_id=row.cellID
        trackid=row.trackID
        f,n=this_id.split('.')
        this_cell=tracked_cells[int(f)-frame_experiment_start][int(n)]
        new_df=pd.DataFrame.from_records([{"cellID":this_id,"trackID":trackid,"cellColor":this_cell['cellColor'],"center":this_cell['center'],"roiPoly":this_cell['roiPoly'],"axis":this_cell['axis']}], index='cellID')
        trap_newDF_list.append(new_df)
    trap_newDF=pd.concat(trap_newDF_list)
    df_lineages_geoms=pd.merge(df_lineages,trap_newDF,on=['cellID','trackID'])
    trap_df_lineages_geoms.append(df_lineages_geoms)
df_lineages_geoms.head(1)

,lineageID,trackID,cellID,motherID,frame,roiID,length,division,state,tracking_score,GFP,DsRed,cellColor,center,roiPoly,axis
0,7,20.068-44.007,20.068,20.068,20,roi_f20_n68_x317_y325,31.99251,0,0,131.316172,354.444,97.979,#2c0740,POINT (317.3729617346758 325.7724344391575),"POLYGON ((323 326, 325 326, 325 327, 328 327, ...",LINESTRING (331.9999999999999 332.163544760007...


## Calculating napari-masks from polys
### If files are not found set: 
#### recalculateRois=True

In [9]:
empty_mask=np.zeros((n_frames,figylim,figxlim))   ### Creates empty matrix array
matrix_shape = empty_mask.shape[1:]
roi_mask=deepcopy(empty_mask)



recalculateRois=False
#recalculateRois=True


trap_roi_masks=[]
trap_cellID_to_lineage=[]
trap_colorsLid=[]
trap_colorsCid=[]
#trap_cellID_to_masks=[]
for ti,trap in enumerate(trap_list):
    cellID_to_lineage={}
    colorsLid={}
    colorsCid={}

    if recalculateRois:
        this_trap_cells=trap_tracked_cells_corrected[ti]
        df_lineages_geoms=trap_df_lineages_geoms[ti]
        nlineages=df_lineages_geoms.lineageID.unique().shape[0]
        cmap = get_cmap_Lineages(nlineages)
        for frame_i,frame in enumerate(frames_list):
            print("Calulating frame: %s"%frame,end="\r")
            this_frame_df_lineages_geoms=df_lineages_geoms[df_lineages_geoms["frame"]==frame]
            this_cells=this_trap_cells[frame_i]
            for this_cell in this_cells:
                cellID=this_cell["cellID"]
                this_cell_df=this_frame_df_lineages_geoms[this_frame_df_lineages_geoms["cellID"]==cellID]
                if(this_cell_df.shape[0]==0):
                    this_lid=nlineages+1
                elif(this_cell_df.shape[0]>1):
                    #display(this_cell_df)
                    this_cell_df2=this_cell_df[this_cell_df.tracking_score==this_cell_df.tracking_score.max()]
                    this_lid_list=list(this_cell_df2.lineageID.unique())
                    if(this_lid_list==[]):
                        #display(this_cell_df)
                        #display(this_cell_df2)
                        this_lid=nlineages+1
                    else:
                        this_lid=int(this_lid_list[0])
                else:#(this_cell_df.tracking_score.isnull()==True):
                    this_lid=int(this_cell_df.lineageID)
                
                this_color=cmap(this_lid)
                colorsLid[this_lid]=this_color
                colorsCid[float(cellID)]=this_cell["cellColor"]
                cellID_to_lineage[float(cellID)]=this_lid

                this_poly=this_cell["roiPoly"]
                napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
                mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
                roi_mask[frame_i][mask] = float(cellID)
        trap_roi_masks.append(roi_mask)   
        print("\nSaving files")
        this_file=dirNameDataNapariLabels+"%s_%s_roi_mask.npy"%(trap,seg_chan)
        np.save(this_file,roi_mask)
        this_file=dirNameDataNapariLabels+"%s_%s_colors_dict.npy"%(trap,seg_chan)
        np.save(this_file,colorsCid)
        this_file=dirNameDataNapariLabels+"%s_%s_colors_lineages_dict.npy"%(trap,seg_chan)
        np.save(this_file,colorsLid)
        this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict.npy"%(trap,seg_chan)
        np.save(this_file,cellID_to_lineage)
        
    else:
        print("Loading files files...")
        this_file=dirNameDataNapariLabels+"%s_%s_roi_mask.npy"%(trap,seg_chan)
        roi_mask=np.load(this_file)
        this_file=dirNameDataNapariLabels+"%s_%s_colors_dict.npy"%(trap,seg_chan)
        colorsCid=np.load(this_file,allow_pickle=True).item()
              
        this_file=dirNameDataNapariLabels+"%s_%s_colors_lineages_dict.npy"%(trap,seg_chan)
        colorsLid=np.load(this_file,allow_pickle=True).item()
        
        this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict.npy"%(trap,seg_chan)
        cellID_to_lineage=np.load(this_file,allow_pickle=True).item()
        this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict_corrected.npy"%(trap,seg_chan)
        if(os.path.exists(this_file)):
            cellID_to_lineage=np.load(this_file,allow_pickle=True).item()
            
        trap_roi_masks.append(roi_mask)
        trap_colorsCid.append(colorsCid)
        trap_colorsLid.append(colorsLid)
        trap_cellID_to_lineage.append(cellID_to_lineage)
        #trap_cellID_to_masks.append(cellID_to_masks)
print("Done!")

Loading files files...
Done!


### Small fix for bug from lineage maker
#### Linegae maker ----> when 2 tracks/lineages colide in the first frame, two lineages are kept

In [24]:
df_lineages_geoms=fix_cells_with_double_lineages(df_lineages_geoms)
xx=df_lineages_geoms.lineageID.unique()
for x in xx:
    df_lineages_geoms=fix_wrong_divisions(df_lineages_geoms,x)


Checking wrong divs for lineage 7
Checking wrong divs for lineage 8
Checking wrong divs for lineage 10
Checking wrong divs for lineage 3
Checking wrong divs for lineage 12
Checking wrong divs for lineage 13
Checking wrong divs for lineage 5
Checking wrong divs for lineage 1
Checking wrong divs for lineage 16
Checking wrong divs for lineage 34
Checking wrong divs for lineage 18
Checking wrong divs for lineage 21
Checking wrong divs for lineage 22
Checking wrong divs for lineage 11
Checking wrong divs for lineage 27
Checking wrong divs for lineage 29
Checking wrong divs for lineage 30
Checking wrong divs for lineage 106
Checking wrong divs for lineage 9
Checking wrong divs for lineage 32
Checking wrong divs for lineage 35
Checking wrong divs for lineage 36
Checking wrong divs for lineage 28
Checking wrong divs for lineage 31
Checking wrong divs for lineage 37
Checking wrong divs for lineage 38
Checking wrong divs for lineage 39
Checking wrong divs for lineage 40
Checking wrong divs for l

Checking wrong divs for lineage 201
Checking wrong divs for lineage 211
Checking wrong divs for lineage 214
Checking wrong divs for lineage 216
Checking wrong divs for lineage 218
Checking wrong divs for lineage 219
Checking wrong divs for lineage 222
Checking wrong divs for lineage 153
Checking wrong divs for lineage 253
Checking wrong divs for lineage 232
Checking wrong divs for lineage 234
Checking wrong divs for lineage 454
Checking wrong divs for lineage 455
Checking wrong divs for lineage 239
Checking wrong divs for lineage 242
Checking wrong divs for lineage 243
Checking wrong divs for lineage 245
Checking wrong divs for lineage 138
Checking wrong divs for lineage 43
Checking wrong divs for lineage 247
Checking wrong divs for lineage 170
Checking wrong divs for lineage 249
Checking wrong divs for lineage 279
Checking wrong divs for lineage 465
Checking wrong divs for lineage 290
Checking wrong divs for lineage 292
Checking wrong divs for lineage 294
Checking wrong divs for linea

## Create local trap data backup for restarting correction session

In [25]:
backup_cells=deepcopy(tracked_cells_corrected)
backup_df_lineages=deepcopy(df_lineages_geoms)
last_save_cells=deepcopy(tracked_cells_corrected)
last_save_df_lineages=deepcopy(df_lineages_geoms)

## Napari viewer

### Open raw/tif images

In [13]:
%gui qt

cellViewer=napari.Viewer()
cellViewer.title="cellViewer-ESB"

trap_index=full_trap_list.index(current_trap)
tif_path=trap_tif_path[trap_index]
masks_path=trap_masks_path[trap_index]

for chan,col,v in zip(channels,channels_color,vis):
    this_path=tif_path+chan+"/"
    cellViewer.open(this_path,blending='additive',colormap=col,name=chan,visible=v)
cellViewer.open(masks_path,blending='additive',colormap='gray',name='masks',visible=False)
cellViewer.dims.set_axis_label(0,"frame(-%s)"%frame_experiment_start)

### Add widgets and buttons

In [14]:
cellViewer.window.remove_dock_widget("all");plotwidget=False

add_lineage_slider_widget(cellViewer)
add_save_buttons(cellViewer)
LoadKeyBindedFuctions(cellViewer)
lineage_slider()
slider_default_val,current_lineageID,current_trackID

Current number of lineages 481
Selecting lineage 1     



(1, 1, 'None')

In [ ]:
#cellViewer.close()


In [ ]:
# cellViewer.window.activate()
# cellViewer.window.qt_viewer.console.execute()

## Fix corrections errors
### Run this after finishing all trap corrections


### Here you get the wrong track names and tracks that were wrongly formed

In [26]:
    
wrong_track_names,wrong_track_names=dataframe_error_reporter(df_lineages_geoms)

Wrong structure tracks []
Wrong named tracks []


### Please corrrect the tracks with wrong structure manually (if any)
#### you can drop them from the dataframe or drop specific lines and re-associate-deassociate

In [27]:
### by cellID
cidx='26.155'
temp_tracks=df_lineages_geoms[df_lineages_geoms["cellID"]==cidx]["trackID"]

### by lineageID

# temp_tracks=df_lineages_geoms[df_lineages_geoms["lineageID"]==15]["trackID"].unique()

### by spicific track
temp_tracks=['cellID1-cellID2']
#temp_tracks=['20.254-41.093']

#temp_tracks.sort()
print(temp_tracks)
for temp_track in temp_tracks:
    display(df_lineages_geoms[df_lineages_geoms["trackID"]==temp_track])


['cellID1-cellID2']


,lineageID,trackID,cellID,motherID,frame,roiID,length,division,state,tracking_score,GFP,DsRed,cellColor,center,roiPoly,axis


In [20]:
#df_lineages_geoms=df_lineages_geoms.drop(4559)


In [29]:
x=max(df_lineages_geoms.lineageID.unique())
nlins=x+1

temp_track=list(temp_tracks)[0]
inxs=df_lineages_geoms[df_lineages_geoms["trackID"]==temp_track].index
print(inxs)
#df_lineages_geoms=df_lineages_geoms.drop(inxs)
#df_lineages_geoms.loc[inxs,"lineageID"]=322
#df_lineages_geoms.loc[4573,"division"]=0
#df_lineages_geoms.loc[inxs,"trackID"]="26.155-41.093"
for temp_track in temp_tracks:
    display(df_lineages_geoms[df_lineages_geoms["trackID"]==temp_track])


Int64Index([], dtype='int64')


,lineageID,trackID,cellID,motherID,frame,roiID,length,division,state,tracking_score,GFP,DsRed,cellColor,center,roiPoly,axis


### This section corrects wrong tracks names
### Set conformation to True to choose correction for each one

In [30]:
df_lineages_geoms=dataframe_error_name_corrrector(df_lineages_geoms,wrong_track_names,confirmation=False)

Automatic changing wrog track names


### This section corrects wrongly splitted tracks upopn division

In [31]:
df_lineages_geoms=fix_splited_tracks_and_divisions(df_lineages_geoms)

checking lineage:
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 60, 61, 63, 64, 66, 67, 68, 69, 
Joining track 20.156-34.066 with track 34.066-36.071
70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 137, 138, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 184, 185, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 2

### check there are no erros left

In [32]:
wrong_track_names2,wrong_track_names2=dataframe_error_reporter(df_lineages_geoms)

Wrong structure tracks []
Wrong named tracks []
Saving corrected tracked cells:
/home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked/xy02/normData/
/home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked_corrected/xy02/normData/
Saving 325 cells to /home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked_corrected/xy02/normData/pBGT-AMP-Pulse_xy02_020.pkl
Saving 302 cells to /home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked_corrected/xy02/normData/pBGT-AMP-Pulse_xy02_021.pkl
Saving 307 cells to /home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked_corrected/xy02/normData/pBGT-AMP-Pulse_xy02_022.pkl
Saving 303 cells to /home/charly/Lab/Projects/uJ/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked_corrected/xy02/normData/pB

### upload session data
### depending on your Gdrive syncer you may want to work locally and then sync to your online data files

In [22]:
rootDir0=uj_data_path_trillion+"pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/"
rootDir1=uj_data_path_trillion_online+"pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/"

dirNameDataCellsTrackedCorrected0=rootDir0+"data_cells_tracked_corrected/"
dirNameDataCellsTrackedCorrected1=rootDir1+"data_cells_tracked_corrected/"
dirNameDataLineagesCorrected0=rootDir0+"data/lineages_corrected/"
dirNameDataLineagesCorrected1=rootDir1+"data/lineages_corrected/"
dirNameDataNapariLabels0=rootDir0+'data_napari_labels/'
dirNameDataNapariLabels1=rootDir1+'data_napari_labels/'


#print("watch -n 600 rsync -auv %s %s | rsync -auv %s %s "%(dirNameDataCellsTrackedCorrected0,dirNameDataCellsTrackedCorrected1,dirNameDataLineagesCorrected0,dirNameDataLineagesCorrected1))
print("Syncing cells")
!rsync -auv $dirNameDataCellsTrackedCorrected0 $dirNameDataCellsTrackedCorrected1 
print("Syncing dataframe")
!rsync -auv $dirNameDataLineagesCorrected0 $dirNameDataLineagesCorrected1
print("Syncing napari labels data")
!rsync -auv $dirNameDataNapariLabels0 $dirNameDataNapariLabels1

Syncing cells
sending incremental file list
xy02/normData/pBGT-AMP-Pulse_xy02_020.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_021.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_022.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_023.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_024.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_025.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_026.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_027.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_028.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_029.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_030.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_031.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_032.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_033.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_034.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_035.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_036.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_037.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_038.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_039.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_040.pkl
xy02/normData/pBGT-AMP-Pulse_xy02_041.pkl
xy02/normData/pBGT-AMP-Pulse_xy0

## Native binded keys
##### Ctrl Switch to fill mode temporarily when pressed
##### Ctrl+Y Toggle ndisplay
##### ← Increment dimensions slider to the left
##### →Increment dimensions slider to the right
##### Alt+↑ Move focus of dimensions slider up
##### Alt+↓ Move focus of dimensions slider down
##### Ctrl+E Change order of the visible axes, e.g. [0, 1, 2] -> [2, 0, 1]
##### Ctrl+T Transpose order of the last two visible axes, e.g. [0, 1] -> [1, 0]
##### Ctrl+Delete Remove selected layers
##### Ctrl+⌫ Remove selected layers
##### Ctrl+A Selected all layers
##### Ctrl+⇧+Delete Remove all layers
##### Ctrl+⇧+⌫ Remove all layers
##### ↑ Select layer above
##### ↓ Select layer below
##### ⇧+↑ Also select layer above
##### ⇧+↓ Also select layer below
##### Ctrl+R Reset view to original state
##### Ctrl+G Toggle grid mode
##### V Toggle visibility of selected layers
##### Space Hold to pan and zoom in the viewer
##### P Activate the paintbrush
##### F Activate the fill bucket
##### Z Activate pan and zoom mode
##### L Activate the label picker
##### E Activate the label eraser
##### M Set the currently selected label to the largest used label plus one
##### D Decrease the currently selected label by one
##### I Increase the currently selected label by one
##### Ctrl+Z Undo the last paint or fill action since the view slice has changed
##### Ctrl+⇧+Z Redo any previously undone actions
##### ⇧ Toggle preserve label option when pressed
##### Alt Switch to erase mode temporarily when pressed
